In [6]:

import easyocr
from PIL import Image
import numpy as np
import pymongo

def image_to_text(path):
        img = Image.open(path)

        # Convert the PIL Image to a numpy array
        image_np = np.array(img)

        reader = easyocr.Reader(['en'])
        text = reader.readtext(image_np, detail=0, paragraph=True)
        return text

def store_mongodb(data):
  gopi = pymongo.MongoClient("mongodb+srv://karthik46:<password>@cluster0.iiqvamg.mongodb.net/")
  db = gopi['bizcardx']
  col = db['image_data']
  col.insert_one(data)

def temp_collection_drop():
    gopi = pymongo.MongoClient("mongodb+srv://karthik46:<password>@cluster0.iiqvamg.mongodb.net/")
    db = gopi['bizcardx']
    col = db.list_collection_names()
    if len(col) > 0:
        for i in col:
            db.drop_collection(i)

def text_to_mongodb(path):
  # extract data from image
  text = image_to_text(path)
  data = {'data':text}

  # delete existing data
  temp_collection_drop()

  # store extracted data
  store_mongodb(data)

text_to_mongodb('/Users/karthikeyank/Documents/biz/3.png')

In [10]:

import easyocr
from PIL import Image
import numpy as np
import pymongo
import sqlite3
import pandas as pd

def image_to_text(path):
        img = Image.open(path)

        # Convert the PIL Image to a numpy array
        image_np = np.array(img)

        reader = easyocr.Reader(['en'])
        text = reader.readtext(image_np, detail=0, paragraph=True)
        return text

def store_mongodb(data):
    df=pd.DataFrame(data)
    conn=sqlite3.connect('bizcardx.db')
    df.to_sql('image_data', conn, if_exists='append', index=False)
    conn.commit()
    conn.close()
    

def temp_collection_drop():
    gopi = pymongo.MongoClient("mongodb+srv://karthik46:<password>@cluster0.iiqvamg.mongodb.net/")
    db = gopi['bizcardx']
    col = db.list_collection_names()
    if len(col) > 0:
        for i in col:
            db.drop_collection(i)

def text_to_mongodb(path):
  # extract data from image
  text = image_to_text(path)
  data = {'data':text}

  # delete existing data
  temp_collection_drop()

  # store extracted data
  store_mongodb(data)

text_to_mongodb('/Users/karthikeyank/āDocuments/biz/2.png')

In [152]:
img = Image.open("/Users/karthikeyank/Documents/biz/2.png")

        # Convert the PIL Image to a numpy array
image_np = np.array(img)

reader = easyocr.Reader(['en'])
text = reader.readtext(image_np, detail=0, paragraph=True)
df=pd.DataFrame(text)


In [153]:
df

,0
0,Amit kumar CEO & FOUNDER
1,123-456-7569
2,hello@global.com WWW global.com 123 global St ...
3,GLOBAL INSURANCE


In [154]:
import pandas as pd
import re

# Assuming `text` contains the extracted text as shown in your previous example
# Create an empty DataFrame with columns for storename, number, email, website, address, and postalcode
df = pd.DataFrame(columns=["storename", "number", "email", "website", "address", "postalcode"])

# Initialize variables
storename = None
number = None
email = None
website = None
address = None
postalcode = None

# Define a regular expression pattern to match 6-digit numbers
zipcode_pattern = r'\b\d{6}\b'

# Iterate through each line of the extracted text
for line in text:
    # Check if the line contains "FULL" in uppercase
    if "FULL" in line.upper():
        storename = line
    # Check if the line contains "+123" or "+91"
    elif "+123" in line or "+91" in line:
        number = line
    # Check if the line contains an email pattern
    elif "@" in line and ".com" in line:
        email = line
    # Check if the line starts with "www." or ends with ".com" or ".in"
    elif line.startswith("www.") or line.endswith(".com") or line.endswith(".in"):
        website = line
    # Check if the line contains a website pattern
    elif re.search(r'\b(?:www\..*\.com|www\..*\.in)\b', line):
        website = line

    # Check if the line looks like an address
    elif any(keyword in line.upper() for keyword in ["ST", "TAMIL NADU"]):
        address = line
    # Check if the line contains a 6-digit postal code
    elif re.search(zipcode_pattern, line):
        postalcode = re.search(zipcode_pattern, line).group()

# Add the extracted data to the DataFrame
df.loc[0] = [storename, number, email, website, address, postalcode]

# Display the resulting DataFrame
df

,storename,number,email,website,address,postalcode
0,None,None,hello@global.com WWW global.com 123 global St ...,None,None,None
